##### Update drivers:

In [ ]:
%%time
!apt update

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (124 kB/s)
Reading package li

##### Install libraries:

In [ ]:
# pdf parsing
!pip3 install tika
#/pdf parsing

# to .docx
!pip3 install python-docx
# to .docx

# to pdf
!pip3 install aspose-words
#/to pdf

# text embeddings
!pip3 install -U sentence-transformers
!pip3 install rouge-score
!pip3 install sacremoses
#/text embeddings

# arxiv
!pip3 install arxiv
#/arxiv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##### Import libraries:

In [ ]:
# general
from scipy import spatial
from random import randint
import itertools
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import spacy, re
import zipfile
import string

import time
import os
import io
#/general

# read pdf
from pathlib import Path
from tika import parser
#/read pdf

# arxiv
import arxiv
#/arxiv

# text preprocessing
from textblob import TextBlob
from spacy.symbols import nsubj, nsubjpass, VERB
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
#/text preprocessing

# text embeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
#/text embeddings

# to docx
import docx
from docx import Document
from docx.enum.text import WD_COLOR_INDEX
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from copy import deepcopy
#/to docx

# to pdf
import aspose.words as aw
#/to pdf

# parallel calculations
tqdm.pandas()
#/parallel calculations

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


##### UDFs:

In [ ]:
def tfidf_sim(x,y):
  
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform([x,y])
  arr = X.toarray()

  return cosine_similarity(arr)[0][1]

def get_rouge(x,y):
  scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
  scores = scorer.score(x,y)

  return round(float(str(scores).split("fmeasure=")[1].split(')}')[0]),2)

def write_docx(title, text):

  document = Document()

  style = document.styles['Normal']
  font = style.font
  font.name = 'Times New Roman'
  font.size = Pt(12)

  paragraph = document.add_paragraph(text)
  paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph.style = document.styles['Normal']

  document.save(title)

def filter_triplet(final_text):
    
    final_text = get_unique_text(final_text)
    doc = spacy_nlp(final_text)
    valid_sents = []

    for s in list(doc.sents):  
        if syntax_full(s):
            valid_sents.append(s.text)
    
    final_text = ' '.join(valid_sents)
    
    return final_text

def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def syntax_full(spacy_sentence):
    result=[]
    for token in spacy_sentence:
        if (token.dep == nsubj or token.dep == nsubjpass) and token.head.pos == VERB:
            result.append(token.head)
    if result:
        return True
    else:
        return False

##### Language models:

In [ ]:
spacy_nlp = spacy.load('en_core_web_sm')
spacy_nlp.max_length = 50000000

##### Parameters:

In [ ]:
reading_time = 10

##### Patterns:

In [ ]:
matches = ["Fig ", "Fig.", "Figure ", "fig ", "figure "]

##### Insert arxiv id:

In [ ]:
arxiv_id = input()

1904.00688


##### Arxiv url:

In [ ]:
arxiv_url = 'https://export.arxiv.org/pdf/' + str(arxiv_id) + '.pdf'

##### Put abstract and conclusions:

In [ ]:
search = arxiv.Search(id_list=[arxiv_id])
paper = next(search.results())
abstract = paper.summary

##### Print summary:

In [ ]:
abstract

'Summaries are important when it comes to process huge amounts of information.\nTheir most important benefit is saving time, which we do not have much\nnowadays. Therefore, a summary must be short, representative and readable.\nGenerating summaries automatically can be beneficial for humans, since it can\nsave time and help selecting relevant documents. Automatic summarization and,\nin particular, Automatic text summarization (ATS) is not a new research field;\nIt was known since the 50s. Since then, researchers have been active to find\nthe perfect summarization method. In this article, we will discuss different\nworks in automatic summarization, especially the recent ones. We will present\nsome problems and limits which prevent works to move forward. Most of these\nchallenges are much more related to the nature of processed languages. These\nchallenges are interesting for academics and developers, as a path to follow in\nthis field.'

##### Extract pdf content:

In [ ]:
%%time
pdf_file = parser.from_file(arxiv_url)
corpus = pdf_file["content"].replace('.\n\n', '.###').replace('?\n\n', '.###').replace('!\n\n', '.###')

2022-09-01 14:42:22,412 [MainThread  ] [INFO ]  Retrieving https://export.arxiv.org/pdf/1904.00688.pdf to /tmp/pdf-1904.00688.pdf.
INFO:tika.tika:Retrieving https://export.arxiv.org/pdf/1904.00688.pdf to /tmp/pdf-1904.00688.pdf.


CPU times: user 145 ms, sys: 23.4 ms, total: 169 ms
Wall time: 26.6 s


##### Filter by triplets:

In [ ]:
%%time
corpus = ' '.join([filter_triplet(i.strip()) for i in tqdm(sent_tokenize(corpus))])

100%|██████████| 1324/1324 [00:11<00:00, 118.34it/s]

CPU times: user 10.8 s, sys: 105 ms, total: 10.9 s
Wall time: 11.3 s


##### Leave passages without figures:

In [ ]:
core_passages = [] 

for j in tqdm(corpus.split('.###')):
  
  if any(x in j for x in matches):
    continue
  if len(sent_tokenize(j.replace('\n', ' '))) >= 3:
      core_passages.append(j.replace('\n', ' '))

df_score = pd.DataFrame(zip(core_passages), columns=['Passage'])
df_score['Abstract'] = str(abstract)

100%|██████████| 200/200 [00:00<00:00, 6012.99it/s]


##### Order by similarity:

In [ ]:
%%time

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

df_score['Passage_Embedding'] = df_score['Passage'].apply(lambda x: model.encode(x))
df_score['Abstract_Embedding'] = df_score['Abstract'].apply(lambda x: model.encode(x))

df_score['Score_Rouge'] = df_score['Passage'].apply(lambda x: get_rouge(x, abstract))
df_score['Score_TFIDF'] = df_score['Passage'].apply(lambda x: tfidf_sim(x, abstract))
df_score['Score_Cos'] = df_score.apply(lambda x: (1-spatial.distance.cosine(x['Abstract_Embedding'], x['Passage_Embedding'])), axis=1)

df_score['Score'] = (df_score['Score_Rouge'] + df_score['Score_Cos'] + df_score['Score_TFIDF'])/3

df_score = df_score.sort_values('Score', ascending=False)
df_score = df_score[df_score['Score'] > 0]
df_score.reset_index(inplace=True)

CPU times: user 3.35 s, sys: 396 ms, total: 3.75 s
Wall time: 5.93 s


##### Extract summary:

In [ ]:
core_passages = []

for i in tqdm(list(df_score['Passage'])):
  if len(word_tokenize(' '.join(core_passages) + ' ' + str(i))) < 238*reading_time:
    core_passages.append(i)

summary = '.\n\n'.join(core_passages) + '.'
summary = summary.strip()

100%|██████████| 125/125 [00:01<00:00, 74.46it/s]


##### Write summaries to docx and pdf:

In [ ]:
write_docx('summ.docx', summary)
doc = aw.Document("summ.docx")
doc.save("summ.pdf")